In [2]:
!pip install openpyxl

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from tvDatafeed import TvDatafeedLive, Interval

# Load environment variables from .env file
load_dotenv()

# Retrieve TradingView credentials from environment variables
username = os.environ.get("TRADINGVIEW_USERNAME")
password = os.environ.get("TRADINGVIEW_PASSWORD")

# Initialize TradingView live data feed
tv = TvDatafeedLive(username, password)

# Define interval dictionary
interval_tf = {
    'M1': Interval.in_1_minute,
    'M5': Interval.in_5_minute,
    'M15': Interval.in_15_minute,
    'H1': Interval.in_1_hour,
    'H4': Interval.in_4_hour,
    'D1': Interval.in_daily
}

# Step 1: Read the Excel file
raw_df = pd.read_excel("maptrade/maptrade.xlsx")

# Convert Time1 and Time2 columns to datetime with the correct format
raw_df["Time1"] = pd.to_datetime(raw_df["Time1"], format="%Y.%m.%d %H:%M:%S")
raw_df["Time2"] = pd.to_datetime(raw_df["Time2"], format="%Y.%m.%d %H:%M:%S")

# Step 2: Find unique symbols
symbols = raw_df["Symbol"].unique()

# Prepare new columns for MaxDDpos and MaxFPpos
raw_df["MaxDDpos"] = None
raw_df["MaxFPpos"] = None

# Define a helper function to get M1 data from TradingView for a given symbol
def get_symbol_data(symbol, timeframe='M1', bars=4500):
    # Fetch historical data for the symbol
    data = tv.get_hist(
        symbol=symbol,
        exchange='ICMARKETS',
        interval=interval_tf[timeframe],
        n_bars=bars
    )
    # Ensure the index is datetime and sorted
    data = data.sort_index()
    return data

# Step 3: Iterate through each symbol, retrieve data, and calculate both MaxDDpos & MaxFPpos
for symbol in symbols:
    # 3.2: Retrieve M1 data for the symbol
    symbol_data = get_symbol_data(symbol, timeframe='M1')

    # Filter trades for that symbol
    symbol_trades = raw_df[raw_df["Symbol"] == symbol]

    # 3.3: Iterate each trade and calculate both columns where applicable
    for idx, trade in symbol_trades.iterrows():
        trade_result = trade["Result"]
        trade_type = trade["Type"]
        start_time = trade["Time1"]
        end_time = trade["Time2"]

        # Slice the symbol_data between start_time and end_time
        period_data = symbol_data.loc[start_time:end_time]

        if period_data.empty:
            continue

        # Determine MaxDDpos (for TP and BE)
        if trade_result in ["TP", "BE"]:
            if trade_type == "buy":
                max_dd = period_data["low"].min()  # For buy: min low
            else:
                max_dd = period_data["high"].max()  # For sell: max high
            raw_df.at[idx, "MaxDDpos"] = max_dd

        # Determine MaxFPpos (for SL and BE)
        if trade_result in ["SL", "BE"]:
            if trade_type == "buy":
                max_fp = period_data["high"].max()  # For buy: max high
            else:
                max_fp = period_data["low"].min()   # For sell: min low
            raw_df.at[idx, "MaxFPpos"] = max_fp

# After processing all symbols and trades, raw_df will have the updated columns
# Step 4: Save the updated DataFrame to a new Excel file
output_file = "maptrade/maptrade_with_maxddpos_maxfppos.xlsx"
raw_df.to_excel(output_file, index=False)

# Print how many rows of data processed
print(f"Processed {len(raw_df)} rows of data and saved to {output_file}")


Processed 191 rows of data and saved to maptrade/maptrade_with_maxddpos_maxfppos.xlsx
